# IMPORT & INITIALISATIONS

In [44]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !python -m spacy download en
# !pip install plotly==5.6.0
# !pip install dash
# !pip install "jupyterlab>=3" "ipywidgets>=7.6"
# !pip install chart-studio==1.1.0
# !pip install psutil

In [1]:
import spacy
import re
import collections
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import Counter

In [2]:
def clean_text(text):
    temp = text.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
#     temp = temp.split()
#     temp = [w for w in temp if not w in stopwords]
#     temp = " ".join(word for word in temp)
    return temp

In [4]:
# Function to get unique words for each aspect 
def unique_words(aspect_dict, aspect): 
    unique_words_dict = {}
    unique_word_list = []
    for desc in aspect_dict[aspect]:
        if desc not in unique_word_list:
            unique_word_list.append(desc)
        
    return unique_word_list

#get the count for each unique word for each aspect 
def get_counts_desc_list(aspect_dict, aspect):
    count_desc = {}
    count_list = []

    
    for desc in aspect_dict[aspect]:
        if desc not in count_desc:
            count_desc[desc] = 1
        else:
            count_desc[desc] += 1 
    
    for desc in count_desc:
        count_list.append(count_desc[desc])

                        
    
    return count_list 


def get_proportion_desc_list(aspect_dict,aspect):
    count_desc = {}
    count_list = []
    proportion_list = []
    total= 0 
    
    
    for desc in aspect_dict[aspect]:
        if desc not in count_desc:
            count_desc[desc] = 1
        else:
            count_desc[desc] += 1 
    
    for desc in count_desc:
        total += count_desc[desc]
    for desc in count_desc:
        proportion_list.append(count_desc[desc]/total)

                        
    
    return proportion_list 
        

In [5]:
nlp = spacy.load("en_core_web_sm")

# GETTING THE DATA

###### MENTAIYA: 
pos_reviews1 = pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\mentaiya_cleaned_pos_reviews.xlsx", names=['Reviews'])
neg_reviews1 = pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\mentaiya_cleaned_neg_reviews.xlsx", names=["Reviews"])

###### FUKUDON

pos_reviews1= pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\fukudon_cleaned_pos_reviews.xlsx", names=['Reviews'])

neg_reviews1= pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\fukudon_cleaned_neg_reviews.xlsx", names=['Reviews'])


###### STICKSNBOWLS 
pos_reviews1 = pd.read_csv(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\stickNbowls_cleaned_pos_reviews.csv", names=['Reviews'])
neg_reviews1 = pd.read_csv(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\stickNbowls_cleaned_neg_reviews.csv", names=["Reviews"])

In [11]:
pos_reviews1 = pd.read_csv(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\stickNbowls_cleaned_pos_reviews.csv", names=['Reviews'])
neg_reviews1 = pd.read_csv(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\stickNbowls_cleaned_neg_reviews.csv", names=["Reviews"])

In [78]:
# VISUALISE YOUR DF HERE 
len(pos_reviews1)

23

In [13]:
neg_reviews1

,Reviews
0,mentaiko fries loklok
1,looking forward try got super shocked outrageo...
2,ordered loklok requested go rice technically l...
3,value money first look bowls look small it’s r...
4,food ok im totally disappointed takeaway nonse...
5,beef bowl unagi toji bowl recently beef slices...
6,ordered mentaiko chicken bowl chicken served t...
7,mentaiko salmon rice bowl way worth generous p...
8,loklok least edible could say way better bedok...
9,try reading online went find shop unfriendly s...


In [14]:
# remove repeats for each df in case have
pos_reviews = pos_reviews1.drop_duplicates("Reviews", inplace = False)
pos_reviews.reset_index(inplace=True)
neg_reviews = neg_reviews1.drop_duplicates("Reviews")
neg_reviews.reset_index(inplace=True)

In [15]:
pos_reviews= pos_reviews.drop(columns="index")
neg_reviews = neg_reviews.drop(columns="index")

# POSITIVE REVIEWS

In [16]:
pos_reviews.tail()

,Reviews
18,bestloklok keep ordering broccoli ended tabaoi...
19,lost life every looked hopeless till chanced u...
20,rice bowl sicck loklok cheap great well
21,mentaiko salmon rice bowl way worth generous p...
22,cheap loklok dollar broccoli damm nice fried s...


In [17]:
# PUT INTO A LIST 
pos_reviews_list = pos_reviews["Reviews"].to_list()

In [37]:
pos_aspect_list = []

for sentence in pos_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = token
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            aspects.append(str(token))
    pos_aspect_list += aspects
    
            
#     print("===========================================")
#     print("************** Aspects **************")
#     print(", ".join(aspects)) #get the noun of sentence
#     print("************** Description ***************")
#     print(descriptive_term)
#     print()

In [49]:
# COUNTING ITEMS 
pos_freq = collections.Counter(pos_aspect_list)
print(pos_freq)
# print("#######################################################################################################################")
top_10_pos= dict(collections.Counter(pos_freq))

Counter({'loklok': 11, 'mentaiko': 11, 'salmon': 9, 'pork': 8, 'bowl': 8, 'rice': 8, 'portion': 5, 'sauce': 5, 'food': 5, 'tasty': 4, 'time': 4, 'belly': 4, 'garlic': 4, 'unagi': 3, 'crispy': 3, 'taste': 3, 'price': 3, 'owner': 3, 'sticks': 2, 'coffee': 2, 'shop': 2, 'bowls': 2, 'dish': 2, 'staff': 2, 'donburi': 2, 'nice': 2, 'prices': 2, 'broccoli': 2, 'toji': 1, 'version': 1, 'takeaway': 1, 'fries': 1, 'purchase': 1, 'wanting': 1, 'sons': 1, 'rest': 1, 'menu': 1, 'leo': 1, 'alvin': 1, 'bro': 1, 'type': 1, 'mentai': 1, 'standard': 1, 'fried': 1, 'ordered': 1, 'spicy': 1, 'option': 1, 'waiting': 1, 'abit': 1, 'amount': 1, 'hawkers': 1, 'great': 1, 'bang': 1, 'buck': 1, 'slices': 1, 'vegetables': 1, 'friends': 1, 'chicken': 1, 'find': 1, 'dons': 1, 'neighbourhood': 1, 'areas': 1, 'orientation': 1, 'recommend': 1, 'style': 1, 'delicious': 1, 'yummy': 1, 'stallholder': 1, 'part': 1, 'owners': 1, 'job': 1, 'visit': 1, 'return': 1, 'experience': 1, 'girl': 1, 'birthday': 1, 'satay': 1, 'gra

In [80]:
top_pos_aspects = []
top_10_pos =dict(top_10_pos)
for key in top_10_pos:
    if top_10_pos[key] >= (len(pos_reviews)*0.1):
        print(key)
        top_pos_aspects.append(key)

unagi
tasty
loklok
crispy
salmon
portion
mentaiko
time
pork
belly
bowl
garlic
rice
taste
price
owner
sauce
food


In [81]:
print(top_pos_aspects)

['unagi', 'tasty', 'loklok', 'crispy', 'salmon', 'portion', 'mentaiko', 'time', 'pork', 'belly', 'bowl', 'garlic', 'rice', 'taste', 'price', 'owner', 'sauce', 'food']


### COMPILING INTO DF - POSITIVE 

Two types of DF created: 
1. Sentence --> Aspect (purpose is to see which sentence pointed to the aspects)
2. Aspect --> Descriptive words (Purpose is to see what each aspect was described)

### Part 1: Create a dictionary that points sentence to aspect then make into a df

In [82]:
# creating a sentence-aspect dictionary 
pos_sent_aspect_dict = {}
for sentence in pos_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = token
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            aspects.append(str(token)) #for each sentence, store all the aspects in a list 
            if str(token) in top_pos_aspects:
                pos_sent_aspect_dict[s] = str(token)    

In [83]:
pos_df = pd.DataFrame.from_dict(pos_sent_aspect_dict, orient='index', columns=['Aspect'])
pos_df

,Aspect
unagi toji,unagi
tasty loklok sticks deep fried version crispy even though takeaway salmon unagi portion good mentaiko fries also remain crispy time,time
wanting try finally sons loved salmon much garlic pork belly awesome going back try rest menu im sure never disappointing leo alvin thanks bunch bro soon,belly
amazing find coffee shop serve different type bowls lokloksalmon mentai bowl good coffee shop standard loklok fried well,loklok
ordered garlic pork belly rice bowl chose spicy option pork belly portion generous overall taste dish fantastic price friendly polite young owner well definitely back try rice bowls loklok,loklok
loklok tasty price reasonable staff friendly well waiting time abit long though,time
amazing salmon mentaiko donburi pleasantly surprised salmon well cooked still tender generous amount mentaiko sauce cooked salmon i ve tried hawkers normally cooked flakythe portion great well great bang buck would ve preferred slices vegetables bowl,bowl
came try friends love mentaiko salmon garlic pork mentaiko chicken food delicious it s nice find adorable dons neighbourhood areas would love come try loklok next time,time
nice foodfreshly donetaste orientation waiting time recommend anyone loves taste new style dish delicious freshly made,time
tasty donburi great prices,tasty


### Part 2: Create a dict then df for aspect-descriptive

In [84]:
# Aim to create a asepct-descriptive term df 

# initialisation
temp_pos_asp_descrp_dict = {}
asp = ''
# code 
for sentence in pos_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''

    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = str(token)
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            asp = str(token)
        
        if asp in top_pos_aspects:
            if descriptive_term != '':
                if asp not in temp_pos_asp_descrp_dict:
                    temp_pos_asp_descrp_dict[asp] = descriptive_term
                else:
                    temp_pos_asp_descrp_dict[asp] += ", " + descriptive_term


In [85]:
pos_aspect_descrp_df = pd.DataFrame.from_dict(temp_pos_asp_descrp_dict, orient='index')

In [86]:
# uncomment if you want to visualise to see output 
# print(pos_asp_descrp_dict)
print(pos_aspect_descrp_df)

                                                          0
portion   good, generous, overall, generous, great, grea...
mentaiko  good, amazing, generous, reasonable, reasonabl...
crispy                                   good, super, super
time                  good, friendly, next, next, donetaste
salmon    much, garlic, amazing, surprised, surprised, s...
pork      garlic, sweet, garlic, cheap, generous, genero...
belly        garlic, garlic, garlic, garlic, garlic, garlic
bowl      different, good, preferred, friendly, crucial,...
loklok    good, young, sweet, strong, cheap, nice, perfe...
taste     overall, nice, strong, strong, strong, strong,...
price     fantastic, friendly, polite, young, reasonable...
owner     young, young, young, young, young, awesome, aw...
rice                              young, reasonable, garlic
sauce     generous, generous, crucial, reasonable, gener...
food      nice, sweet, sweet, sweet, reasonable, good, r...
unagi                                 sw

# POSITIVE VISUALISATION
Use plotly:https://plotly.com/python/getting-started/ 

For each aspect, get unique description words (x-axis) and the count(y-axis).

In [87]:
# Put description as a list for each aspect in top_5_neg_aspects:
pos_asp_dict = {}
for asp in temp_pos_asp_descrp_dict:
    content = temp_pos_asp_descrp_dict[asp]
    to_list = content.split(',')
    pos_asp_dict[asp] = to_list

# #combine 2 similar words
# pos_asp_dict['mentaiko'] += pos_asp_dict['mentai']
# del pos_asp_dict['mentai'] #delete the one added 

In [88]:
for asp in pos_asp_dict:
    x_list = []
    y_list = []
    for x,y in zip(unique_words(pos_asp_dict,asp),get_counts_desc_list(pos_asp_dict, asp)):
        if y>1:
            x_list.append(x)
            y_list.append(y)
    if len(x_list)!=0:
        fig = px.bar(x= x_list, y= y_list, title="Positive: " + asp, labels=dict(x ="Frequency",y= "Description"))
        fig.update_layout(xaxis={'categoryorder':'total descending'})
        fig.show()

In [26]:
for asp in pos_asp_dict:
    df = pd.DataFrame(unique_words(pos_asp_dict,asp), columns=['Description'])
    df['counts'] = get_counts_desc_list(pos_asp_dict, asp)
    fig = px.pie(df, values= 'counts', names='Description', title='Positive: '+ asp)
    fig.show()

# NEGATIVE REVIEWS

In [91]:
neg_reviews_list = neg_reviews["Reviews"].to_list()  #convert to list 

In [92]:
neg_aspect_list = []
neg_appended = []
# sentence_list = []
# a_list = []
for sentence in neg_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = token
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            aspects.append(str(token))
    neg_appended.append(aspects)        

    neg_aspect_list += aspects
    

# UNCOMMENT THE FOLLOWING TO SEE ASPECTS AND DESCRIPTION OUTPUTS
            
#     print("===========================================")
#     print("************** Aspects **************")
#     print(", ".join(aspects)) #get the noun of sentence
#     print("************** Description ***************")
#     print(descriptive_term)
#     print()

##### FOR COUNTING OF ITEMS:
    
Please check and see if there are common words that mean the same thing in the top 5. If yes, then change int in most_common to 6 and combine the contents later below!!!! Else, **use most_common(5)**.

In [ ]:
top_pos_aspects = []
top_10_pos =dict(top_10_pos)
for key in top_10_pos:
    if top_10_pos[key] >= (len(pos_reviews)*0.1):
        print(key)
        top_pos_aspects.append(key)

In [93]:
# COUNTING ITEMS 
neg_freq = collections.Counter(neg_aspect_list)
# top_neg= collections.Counter(neg_freq) #CHANGE HERE IF NEED 
# print(neg_freq, end='')
# top_neg

In [95]:
top_neg_aspects = []
top_neg_dict =dict(neg_freq)
for key in top_neg_dict:
    if top_neg_dict[key] >= (len(neg_reviews)*0.1):
        print(key)
        top_neg_aspects.append(key)

mentaiko
pork
bowl
egg
loklok
rice
items
sauce
broccoli
price
unagi
toji
belly
owner
end
food
sticks
doggy
bag
business
beef
tender
orders
chicken
return
store
serve
portion


## COMPILING INTO DF - NEGATIVE

#### Part 1

In [97]:
neg_sent_aspect_dict = {}
for sentence in neg_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = token
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            aspects.append(str(token)) #for each sentence, store all the aspects in a list 
            if str(token) in top_neg_aspects:
#                 if str(token) not in sent_aspect_dict:
                neg_sent_aspect_dict[s] = str(token)
#                 else:
#                     sent_aspect_dict[str(token)] += s 
#     neg_appended.append(aspects)        

In [98]:
neg_df = pd.DataFrame.from_dict(neg_sent_aspect_dict, orient='index', columns=['Aspect'])

In [99]:
neg_df

,Aspect
mentaiko fries loklok,mentaiko
looking forward try got super shocked outrageous shaoxing braised pork bowl expected egg per picture menu got meatrice couldve gotten caifan stall w ricemeatothers going go back,egg
ordered loklok requested go rice technically loklok rice bowl see pictureall items well fried goes well sauce broccoli well fried especially rice well cooked too suggest something sourish offset jelatness fried items maybe lemon sauce pickled radish ginger would good variety loklok more price reasonable loklok,loklok
value money first look bowls look small it s really packed goodness start eating portions nicei ordered unagi toji kicap manis pork belly bowl suggestion owner might want reduce sauce rice gets salty towards end,end
food ok im totally disappointed takeaway nonsense u charge additional sticks loklok doggy bag regular buying loklok snack food everytime nearby usually sticks yesterday bought bcos selections wasnt u cheek charge additional doggy bag wasnt even charged past mo ths shameful business like thatup loklok price decent business charging doggy bag like leaves distaste mouth,bag
beef bowl unagi toji bowl recently beef slices tough unagi tender either wait peak hours staff handling orders customer orders seems long,orders
ordered mentaiko chicken bowl chicken served totally uncook inside return store took super long serve asked even reprepared bad apologetic asked refund end beef mushroom bowl well recommend,bowl
mentaiko salmon rice bowl way worth generous portion came mentaiko sauce owner also gave free onsen egg far also tried garlic pork belly rice bowl cheap portion garlic pork generous also sell loklok stick like brocolli loklok best also sauce used totally recommend ppl tampines give try least,sauce
loklok least edible could say way better bedok loklok frankly speaking limited edition choose overall would grade,loklok
ordered mentaiko chicken bowl chicken served uncook fr inside return store took super long serve asked even reprepared bad recommend,serve


#### Part 2

In [101]:
# Aim to create a asepct-descriptive term df 

# initialisation
temp_neg_asp_descrp_dict = {}

# code 
for sentence in neg_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = str(token)
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            asp = str(token)
            aspects.append(str(token)) #for each sentence, store all the aspects in a list 
            
        if asp in top_neg_aspects:
            
            if descriptive_term != '':
                
                if asp not in temp_neg_asp_descrp_dict:
                    temp_neg_asp_descrp_dict[asp] = descriptive_term
                    
                else:
                    temp_neg_asp_descrp_dict[asp] += ',' + descriptive_term


In [102]:
neg_asp_descrp_df = pd.DataFrame.from_dict(temp_neg_asp_descrp_dict, orient='index',columns=['Description'])
neg_asp_descrp_df

,Description
pork,"outrageous,packed,garlic,cheap,generous,genero..."
bowl,"outrageous,outrageous,packed,bad,bad,bad,garli..."
egg,"outrageous,outrageous,free,free,free,free,garlic"
items,"fried,fried,fried,sourish,sourish,sourish"
sauce,"fried,sourish,sourish,packed,generous,generous..."
broccoli,"fried,fried,fried,fried"
rice,"fried,fried,fried,fried,fried,fried,sourish,so..."
price,"more,reasonable,doggy,decent"
loklok,"reasonable,regular,generous,generous,generous,..."
unagi,packed


# NEGATIVE VISUALISATION
Top 5 for NEG

Use plotly ==> https://plotly.com/python/getting-started/ 
For each aspect, get unique description words (x-axis) and the count(y-axis).

In [103]:
# Put description as a list for each aspect in top_5_neg_aspects:
neg_asp_dict = {}
for asp in temp_neg_asp_descrp_dict:
    content = temp_neg_asp_descrp_dict[asp]
    to_list = content.split(',')
    neg_asp_dict[asp] = to_list

# #combine 2 similar words HERE
# neg_asp_dict['mentaiko'] += neg_asp_dict['mentai']
# del neg_asp_dict['mentai']

#### Frequency Visualisation

In [104]:
for asp in neg_asp_dict:
    x_list = []
    y_list = []
    for x,y in zip(unique_words(neg_asp_dict,asp),get_counts_desc_list(neg_asp_dict, asp)):
        if y>1:
            x_list.append(x)
            y_list.append(y)
    if len(x_list)!=0:
        fig = px.bar(x= x_list, y= y_list, title="Negative: " + asp, labels=dict(x ="Description",y= "Frequency"))
        fig.update_layout(xaxis={'categoryorder':'total descending'})
        fig.show()

#### Proportion Visualisation

In [74]:
for asp in neg_asp_dict:
    df = pd.DataFrame(unique_words(neg_asp_dict,asp), columns=['Description'])
    df['counts'] = get_counts_desc_list(neg_asp_dict, asp)
    fig = px.pie(df, values= 'counts', names='Description', title='Negative: '+ asp)
    fig.show()

# DOCUMENTATION FROM ONLINE (IGNORE)

In [ ]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ == 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  print(sentence)
  print(descriptive_term)

In [ ]:
aspects = []
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  target = ''
  for token in doc:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      target = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  aspects.append({'aspect': target,
    'description': descriptive_term})
print(aspects)

### Conduct part of speech tags 

In [22]:
for s in sentences:
    doc = nlp(s)
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
      token.pos_,[child for child in token.children]) 

The det food NOUN DET []
food nsubj was AUX NOUN [The, had]
we nsubj had VERB PRON []
had relcl food NOUN VERB [we, yesterday]
yesterday npadvmod had VERB NOUN []
was ROOT was AUX AUX [food, delicious]
as advmod delicious ADJ ADV []
delicious acomp was AUX ADJ [as, as]
as prep delicious ADJ ADP [one]
the det one NOUN DET []
one pobj as ADP NOUN [the, had]
we nsubj had VERB PRON []
had relcl one NOUN VERB [we, in]
in prep had VERB ADP [Italy]
Italy pobj in ADP PROPN []
Although mark was AUX SCONJ []
the det meal NOUN DET []
meal nsubj was AUX NOUN [the]
was ROOT was AUX AUX [Although, meal, tasty, ,, but, was]
very advmod tasty ADJ ADV []
tasty acomp was AUX ADJ [very]
, punct was AUX PUNCT []
but cc was AUX CCONJ []
the det internet NOUN DET []
internet nsubj was AUX NOUN [the]
was conj was AUX AUX [internet, slow, .]
slow acomp was AUX ADJ []
. punct was AUX PUNCT []
Fukudon nsubj was AUX PROPN []
was ROOT was AUX AUX [Fukudon, weird]
weird acomp was AUX ADJ []


### Pick up sentiment description first 

In [23]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      descriptive_term = token
  print(sentence)
  print(descriptive_term)

The food we had yesterday was as delicious as the one we had in Italy
delicious
Although the meal was very tasty, but the internet was slow.
slow
Fukudon was weird
weird


In [25]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ == 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  print(sentence)
  print(descriptive_term)

The food we had yesterday was as delicious as the one we had in Italy
as delicious
Although the meal was very tasty, but the internet was slow.
slow
Fukudon was weird
weird


In [26]:
aspects = []
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  target = ''
  for token in doc:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      target = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  aspects.append({'aspect': target,
    'description': descriptive_term})
print(aspects)

[{'aspect': 'food', 'description': 'as delicious'}, {'aspect': 'internet', 'description': 'slow'}, {'aspect': '', 'description': 'weird'}]


In [28]:
pip install TextBlob

     ------------------------------------- 636.8/636.8 KB 10.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [29]:
from textblob import TextBlob
for aspect in aspects:
  aspect['sentiment'] = TextBlob(aspect['description']).sentiment
print(aspects)

[{'aspect': 'food', 'description': 'as delicious', 'sentiment': Sentiment(polarity=1.0, subjectivity=1.0)}, {'aspect': 'internet', 'description': 'slow', 'sentiment': Sentiment(polarity=-0.30000000000000004, subjectivity=0.39999999999999997)}, {'aspect': '', 'description': 'weird', 'sentiment': Sentiment(polarity=-0.5, subjectivity=1.0)}]


# THE END 